# 深度学习实践作业 2-de  
## 生成对抗网络 (Generative Adversarial Networks, GANs)

**简介**  
到目前为止，我们已经通过分类模型研究了计算机视觉中的“判别式”方法。从统计学的角度来看，这些方法建模的是条件分布 $ P(Y | X) $。在本次实践中，我们将探讨生成式模型，这类模型传统上建模联合分布 $ P(X, Y) $，从而可以推断 $ X $ 的似然性。特别地，我们将研究生成对抗网络（Generative Adversarial Networks, GANs）。虽然它们并不完全符合传统生成式模型的定义，但却非常接近。自2014年首次提出以来，GANs 在无监督学习中产生了巨大影响，因为它们几乎不需要标签或只需要极少的标签。GANs 建模数据底层结构的能力使其成为图像生成、补全和编辑等领域前沿方法的核心。例如，你能猜到图1中的所有人脸都是假的吗？！

“经典” GAN 的目标是能够从预定义分布中采样的噪声向量 $ z $ 生成一幅图像 $ \tilde{x} $：  
$$
\tilde{x} = G(z), \, z \sim P(z)
$$  

我们可以很容易将这一框架扩展为条件 GAN（conditional GAN, cGAN），其目标是根据噪声向量 $ z $ 和输入 $ y $（指导生成过程）生成图像 $ \tilde{x} $。例如，$ y $ 可以是一个类别标签、一幅图像、一句话等。  
$$
\tilde{x} = G(z, y), \, z \sim P(z)
$$  

### **第1部分 – 生成对抗网络**  
**1.1 基本原理**  
首先，我们将研究“经典的”无条件 GAN（Goodfellow 等人，2014）。具体来说，我们将以 DCGAN（Radford 等人，2016）架构为基础，因为它是当前大多数前沿 GAN 模型的基础。  

GAN 的目标是生成逼真的数据，即符合真实数据分布 $ P(X) $ 的可能数据。然而，这个分布显然是未知的：我们只有从该分布中采样得到的数据样本，即数据集中的示例：  
$$
\text{Data} = \{x_i \sim P(X)\}_{i=1..N}
$$  

**生成器**  
为了解决该任务，我们定义了第一个神经网络 $ G $。它的目标是将从固定分布（通常为 $ U_{[-1,1]} $ 或 $ \mathcal{N}(0, I) $）中采样的任意输入 $ z $ 转换为逼真的图像 $ \tilde{x} $。  

$$
\tilde{x} = G(z), \quad z \sim P(z) \tag{3}
$$  

**判别器**  
用于训练生成器的损失函数并不简单，因为分布 $ P(X) $ 是未知的。为了解决这个问题，我们提出训练第二个神经网络：判别器 $ D $。该网络以图像作为输入，并预测这是否是真实图像 $ x^* $（来自训练数据）或生成器生成的虚假图像 $ \tilde{x} $。  

$$
D(x) \in [0,1], \quad \text{理想情况下，} \quad
\begin{cases}
D(\tilde{x}) = 0, & \tilde{x} = G(z) \\  
D(x^*) = 1, & x^* \in \mathcal{D}ata  
\end{cases}
$$  

**对抗训练**  
训练 GAN 是一种特殊的过程，可以被视为一个“博弈”（基于博弈论的概念），其中两个对手相互竞争：生成器不断学习以欺骗判别器，而判别器则不断适应生成器的变化以区分真实样本和虚假样本。判别器的分类损失为二元交叉熵损失。  

我们希望优化以下问题：  

$$
\min_G \max_D \mathbb{E}_{x^* \in \mathcal{D}ata} \left[ \log D(x^*) \right] + \mathbb{E}_{z \sim P(z)} \left[ \log(1 - D(G(z))) \right] \tag{5}
$$  

具体而言，从实际操作的角度来看，我们在训练生成器和判别器之间交替进行，它们各自有一个从上述方程推导出的不同目标函数。  

$$
\max_{G} \mathbb{E}_{z \sim P(z)}[\log D(G(z))] \tag{6}
$$  

$$
\max_{D} \mathbb{E}_{x^* \in \mathcal{D}ata}[\log D(x^*)] + \mathbb{E}_{z \sim P(z)}[\log(1 - D(G(z)))] \tag{7}
$$  

**问题**  

1. 解释方程 (6) 和 (7)。如果只使用其中一个会发生什么？  
2. 理想情况下，生成器 $ G $ 应将分布 $ P(z) $ 转换为什么？  
3. 注意到方程 (6) 并非直接从方程 (5) 推导而来。作者通过这种方式使训练更加稳定，并避免梯度饱和现象。这里“真实的”方程应该是什么？  

**1.2 网络架构**  

大多数用于图像生成的 GAN 模型基于 Radford 等人（2016）提出的深度卷积 GAN (DCGAN) 架构，如图 2 所示。该架构为判别器和生成器提供了基于卷积层、批归一化层 (Batch Normalization)、ReLU 激活函数的设计，并包含若干技巧和训练指导，以实现图像生成。在我们的案例中，我们将使用一个变体来生成尺寸为 $32 \times 32$ 像素的图像。  

**判别器**  
判别器的架构是图像“分类”模型的经典设计。它使用由卷积层、批归一化层和 LeakyReLU 激活函数组成的模块。在实现设计中，包括以下技巧：用步幅为 2 的卷积代替池化操作，使用大小为 4 的卷积核，以及使用 LeakyReLU 激活函数；目的是更好地传递梯度。具体的架构如下：  

- 卷积层（32 个滤波器，卷积核大小为 4，步幅为 2，填充为 1，无偏置）+ 批归一化 + LeakyReLU ($\alpha = 0.2$)  
- 卷积层（64 个滤波器，卷积核大小为 4，步幅为 2，填充为 1，无偏置）+ 批归一化 + LeakyReLU ($\alpha = 0.2$)  
- 卷积层（128 个滤波器，卷积核大小为 4，步幅为 2，填充为 1，无偏置）+ 批归一化 + LeakyReLU ($\alpha = 0.2$)  
- 卷积层（1 个滤波器，卷积核大小为 4，步幅为 1，填充为 0，无偏置）+ Sigmoid 激活函数  

**生成器**  
对于生成器，我们希望从一个没有空间信息的向量 $z$ 开始，并逐步注入空间信息，最终生成目标图像。为此，核心思想是使用卷积的“逆”操作，这种操作可以将例如尺寸为 $4 \times 4 \times 64$ 的张量转换为尺寸为 $8 \times 8 \times 32$ 的张量。这种操作称为转置卷积（也称为逆卷积或反卷积），其功能在图 3 中描述。基本思想是在应用卷积之前，虚拟地“展开”特征图的“像素”（从而在空间上增加图像的尺寸）。  

除了使用这种特殊的层外，网络的设计相对简单，其具体架构如下：  

- 转置卷积（128 个滤波器，卷积核大小为 4，步幅为 1，填充为 0，无偏置）+ 批归一化 + ReLU  
- 转置卷积（64 个滤波器，卷积核大小为 4，步幅为 2，填充为 1，无偏置）+ 批归一化 + ReLU  
- 转置卷积（32 个滤波器，卷积核大小为 4，步幅为 2，填充为 1，无偏置）+ 批归一化 + ReLU  
- 转置卷积（3 个滤波器，卷积核大小为 4，步幅为 2，填充为 1，无偏置）+ Tanh 激活函数  


**实现**  
按照提供的 notebook 中的指示操作。你将需要完成以下部分：  
- 定义两个网络的架构。  
- 定义我们将使用的先验分布 $ P(z) $。这里我们选择高斯先验分布 $ \mathcal{N}(0, I) $，其维度为 $ nz $（一个可以调整的超参数，默认为 100）。  
- 判别器“分类”的损失函数，使用二元交叉熵损失 (binary cross-entropy loss)。  
- 为每个网络定义优化器。我们将使用 Adam 优化器。  
- 为网络编写训练代码。  

在 notebook 中默认给出的超参数设置下，你的 GAN 应能够生成数字。在经过几百个批次后，生成的图像应逐渐成形；在 1500 个批次后，生成的图像应开始看起来较为逼真。  

当代码可以正常工作后，建议你研究与 GAN 相关的不同方面，观察这种架构的功能和局限性。以下是一些可能的测试：  
- 修改 $ ngf $ 或 $ ndf $，特别是显著增大或减小其中一个。  
- 将自定义的权重初始化替换为 PyTorch 的默认初始化。  
- 将生成器的训练损失替换为从原始方程（参见问题 3）推导出的“真实”损失。  
- 修改一个或两个模型的学习率。  
- 增加训练时间（例如：训练 30 个 epoch），即使模型似乎已经生成了不错的图像。  
- 显著减小或增大 $ nz $ 的值（例如：$ nz = 10 $ 或 $ nz = 1000 $）。  
- 使用训练好的 GAN，从两个噪声向量 $ z_1 $ 和 $ z_2 $ 中生成图像，尝试生成对应于线性插值 $ \alpha z_1 + (1 - \alpha) z_2, \alpha \in [0, 1] $ 的图像。  
- 尝试通过在模型中添加一个新的模块生成 $ 64 \times 64 $ 的图像（定义数据加载器时相应调整真实数据的大小）。  
- 尝试使用其他数据集，例如 CelebA 或 CIFAR-10。  

**问题**  
4. 评论使用默认设置训练 GAN 的情况（生成过程的进展、损失、稳定性、图像多样性等）。  
5. 评论你根据上述建议进行的不同实验。特别是，评论训练的稳定性、损失、生成图像的多样性等。  

### **第2部分 – 条件生成对抗网络**  
**2.1 一般原理**  
如果生成器和判别器通过一个额外的条件 \(y\) 来进行条件化，GAN 可以扩展为条件模型（Mirza & Osindero, 2014）。例如，在 MNIST 数据集中，我们可以将类别标签输入网络。在人脸生成的情况下，我们可以提供一些属性信息（例如，戴眼镜、头发颜色等）。这个条件也可以是文本或图像。你可以使用这个酷炫的工具，通过提供草图、分割图或文本描述（或三者兼有）来玩转最先进的条件 GAN。请参见图 4a 和 4b。  
在我们的案例中，我们的网络将被定义如下：  

**生成器**  
条件生成器 $ cG(z, y) $ 将从噪声向量 $ z $ 和与图像 $ x^* \in \mathcal{D}ata $ 相关联的属性向量 $ y $ 生成图像。

$$
\tilde{x} = cG(z, y), \quad z \sim P(z), \quad y = \text{attribute}(x^*) \tag{8}
$$

**判别器**  
为了训练我们的条件生成器，我们定义了条件判别器 $ cD(x, y) $，它预测图像 $ x $ 是否具有属性 $ y $，并判断该图像是来自真实数据集 $ x^* $ 还是由生成器生成的 $ \tilde{x} $。

$$
D(x, y) \in [0,1], \quad \text{理想情况下，} \quad
\begin{cases}
D(\tilde{x}, y) = 0, & \tilde{x} = G(z, y) \\
D(x^*, y) = 1, & x^* \in \mathcal{D}ata
\end{cases}
$$

**2.2 cDCGAN 架构用于 MNIST**  
我们将扩展之前的架构，以适应一个额外的条件：我们希望生成或区分的数字的标签。在这里，$x$ 是图像，$y$ 是关联的标签。$y$ 将表示为一个大小为 10 的 one-hot 向量（例如：3 → [0,0,0,1,0,0,0,0,0]）。我们的目标是训练一个生成器，该生成器以噪声向量 $z$ 和标签 $y$ 作为输入，生成一个大小为 $32 \times 32$ 的图像 $ \hat{x} $，该图像与标签 $y$ 相对应。  

我们将继续使用我们的生成器和判别器模块。然而，我们需要一种方法，将标签与噪声向量或图像进行融合，然后再应用标准的 DCGAN 架构。图 5 详细描述了融合过程。

**判别器**  
我们的判别器与之前非常相似，但我们需要将图像与 one-hot 向量的数字信息进行融合。完整的架构描述如下：

**分支 1：图像 $x$ 的投影**  
— 卷积（64 个滤波器，核大小 4，步长 2，填充 1）+ LeakyReLU（$\alpha$ = 0.2）  

**分支 2：标签 $y$ 的投影**  
— 将 $y$ 扩展为一个大小为 $10 \times 32 \times 32$ 的张量  
— 卷积（64 个滤波器，核大小 4，步长 2，填充 1）+ LeakyReLU（$\alpha$ = 0.2）  

**连接与分类**  
— 将分支 1 和分支 2 拼接（64 + 64 个通道）  
— 卷积（256 个滤波器，核大小 4，步长 2，填充 1，无偏置）+ Batch Norm + LeakyReLU（$\alpha$ = 0.2）  
— 卷积（512 个滤波器，核大小 4，步长 2，填充 1，无偏置）+ Batch Norm + LeakyReLU（$\alpha$ = 0.2）  
— 卷积（1 个滤波器，核大小 4，步长 1，填充 0）  
— Sigmoid 激活  

**生成器**  
同样，生成器需要将标签 $y$ 与噪声向量进行融合。我们执行与之前相似的拼接操作，详细描述如下：

**分支 1：向量 $z$ 的投影**  
— 反卷积（256 个滤波器，核大小 4，步长 1，填充 0，无偏置）+ Batch Norm + ReLU  

**分支 2：标签 $y$ 的投影**  
— 反卷积（256 个滤波器，核大小 4，步长 1，填充 0，无偏置）+ Batch Norm + ReLU  

**连接与图像生成**  
— 将分支 1 和分支 2 拼接（256 + 256 个通道）  
— 反卷积（256 个滤波器，核大小 4，步长 2，填充 1，无偏置）+ Batch Norm + ReLU  
— 反卷积（128 个滤波器，核大小 4，步长 2，填充 1，无偏置）+ Batch Norm + ReLU  
— 反卷积（1 个滤波器，核大小 4，步长 2，填充 1）  
— Tanh

**实践**  
按照提供的 notebook 中的指示操作（完成 ConditionalGenerator 和 ConditionalDiscriminator 类，并编写训练所需的代码）。  
使用 notebook 中提供的默认超参数。图像应该很快开始形成，并且在 2000 个批次后应该看起来几乎完美。

**问题**  
6. 请评论你使用条件 DCGAN 的经验。  
7. 我们可以从判别器的输入中移除向量 $y$ 吗（即使用 $cD(x)$ 而不是 $cD(x, y)$）？  
8. 你的训练是否比无条件的情况更成功？为什么？  
9. 在代码的最后进行测试。每一列对应一个独特的噪声向量 $z$。在这里，$z$ 可以被解释为什么？